<a href="https://colab.research.google.com/github/thinking-machine12/Colab-File/blob/main/AdultingChat_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Brief:** The main purpose of the chatbot is to guide users on the requirements needed to be able to get a valid ID and/or open a bank account. In the beginning of the chat, the user shall indicate the intention whether they are seeking to open a bank account or get a valid ID. The response of the bot will depend on the user selection. After this, the user will indicate what type of ID or which bank they are interested. Based on the selection, the bot can give answers on general information, process overview, eligibility, validity, and requirements.

**Step 0: Create a Project Directory**

**Step 1: Set Up Your RASA Environment**

In [129]:
!pip install rasa

In [130]:
!rasa init --no-prompt

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe

**Step 2: Define intents and entities**

In [131]:
!pip install pandas pyyaml openpyxl

In [132]:
import pandas as pd
import yaml

In [134]:
nlu_df = pd.read_excel("/content/PH Adulting Dataset.xlsx", sheet_name="questions")

In [133]:
nlu_df = pd.read_excel("/content/PH Adulting Dataset.xlsx", sheet_name="questions")

FileNotFoundError: [Errno 2] No such file or directory: 'your-credentials.json'

In [135]:
nlu_df

,text,intent
0,hi,greet
1,hi there,greet
2,hello,greet
3,hello there,greet
4,hey,greet
...,...,...
1565,Tell me about the SSS UMID.,inform_ID
1566,I need to open a bank account.,inform_bank
1567,I'm interested in opening an account with BDO.,inform_bank
1568,Can you help me with Metrobank?,inform_bank


In [148]:
# Load the Excel file
file_path = '/content/PH Adulting Dataset.xlsx'
sheet_name = 'questions'

# Read the Excel sheet into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Initialize a dictionary to store the intents and examples
intents = {}

# Iterate through the DataFrame and populate the intents dictionary
for index, row in df.iterrows():
    intent = row['intent']
    example = row['text']

    if intent not in intents:
        intents[intent] = []

    intents[intent].append(example)

# Create a dictionary in the format required for the YAML file
nlu_data = {'version': '2.0', 'nlu': []}

for intent, examples in intents.items():
    nlu_data['nlu'].append({
        'intent': intent,
        'examples': '\n'.join([f'- {example}' for example in examples])
    })

# Write the dictionary to a YAML file
with open('/content/data/nlu.yml', 'w') as file:
    yaml.dump(nlu_data, file, sort_keys=False)

print(f"YAML file '/content/data/nlu.yml' has been created successfully.")

YAML file '/content/data/nlu.yml' has been created successfully.


**Step 3: Create Responses Data**

In [138]:
responses_df = pd.read_excel("/content/PH Adulting Dataset.xlsx", sheet_name="responses")

In [139]:
responses_df

,response,text
0,utter_greet,Hello! Welcome to adulting! My name is Bridget...
1,utter_ask_name,"To get started, may I know your name?"
2,utter_name,"Nice to meet you, (name)! How can I help?"
3,utter_bot_challenge,I am a bot.
4,utter_further_assistance,Is there anything else I can help you with?
...,...,...
137,utter_bankAccount_need,"No, you do not need to have a bank account whe..."
138,utter_savingsAccount_info,A regular savings account is a more common typ...
139,utter_savingsAccount_need,"No, you do not need to have a savings account ..."
140,utter_bank_list,"Currently, I can provide information for the t..."


In [141]:
new_responses_yml = {}
new_responses_yml['version'] = '2.0'

responses_list = []

# Iterate through all intents
for response in responses_df['response'].unique():

    response_dict = {}

    response_dict['response'] = str(response).strip()

    tmp_df = responses_df[responses_df['response'] == response]
    print("Found response: ", response)

    example_txt = ""
    for index, row in tmp_df.iterrows():
        tmp_txt = row['text']
        print("Found text: ", tmp_txt)

        # Ensure tmp_txt is a string and handle None values
        if tmp_txt is not None:
            tmp_txt = str(tmp_txt)
            tmp_txt = "".join(["- ", tmp_txt.strip(), "\n"])
            example_txt = "".join([example_txt, tmp_txt])

    response_dict['examples'] = example_txt
    responses_list.append(response_dict)

new_responses_yml['responses'] = responses_list

Found response:  utter_greet
Found text:  Hello! Welcome to adulting! My name is Bridget, and I am your adulting assistant. I can assist you with getting an ID and opening a bank account.
Found response:  utter_ask_name
Found text:  To get started, may I know your name?
Found response:  utter_name
Found text:  Nice to meet you, (name)! How can I help?
Found response:  utter_bot_challenge
Found text:  I am a bot.
Found response:  utter_further_assistance
Found text:  Is there anything else I can help you with?
Found response:  utter_goodbye
Found text:  Hope I was able to help. Bye, bye!
Found response:  utter_out_of_bounds
Found text:  I'm sorry! We don't have information on that specific topic. You can ask me about the following:

1. First Time Job Seeker Assistance
2. Onboarding Requirements
2.1. NBI Clearance
2.2. Police Clearance
2.3. Medical Certificate
2.4. PSA Birth Certificate
2.5. SSS Number
2.6. PhilHealth Number
2.7. PagIBIG Number
2.8. TIN
2.9. Transcript of Records
2.10. D

In [142]:
new_responses_yml

{'version': '2.0',
 'responses': [{'response': 'utter_greet',
   'examples': '- Hello! Welcome to adulting! My name is Bridget, and I am your adulting assistant. I can assist you with getting an ID and opening a bank account.\n'},
  {'response': 'utter_ask_name',
   'examples': '- To get started, may I know your name?\n'},
  {'response': 'utter_name',
   'examples': '- Nice to meet you, (name)! How can I help?\n'},
  {'response': 'utter_bot_challenge', 'examples': '- I am a bot.\n'},
  {'response': 'utter_further_assistance',
   'examples': '- Is there anything else I can help you with?\n'},
  {'response': 'utter_goodbye',
   'examples': '- Hope I was able to help. Bye, bye!\n'},
  {'response': 'utter_out_of_bounds',
   'examples': "- I'm sorry! We don't have information on that specific topic. You can ask me about the following:\n\n1. First Time Job Seeker Assistance\n2. Onboarding Requirements\n2.1. NBI Clearance\n2.2. Police Clearance\n2.3. Medical Certificate\n2.4. PSA Birth Certif

In [143]:
with open("new_responses.yaml", 'w') as file:
    tmp = yaml.dump(new_responses_yml, file)

In [144]:
# Let's see how it will look like if we read it as a yaml file

with open("new_responses.yaml", 'r') as file:
    new_responses_yaml_tmp = yaml.load(file, Loader=yaml.FullLoader)

new_responses_yaml_tmp

{'responses': [{'examples': '- Hello! Welcome to adulting! My name is Bridget, and I am your adulting assistant. I can assist you with getting an ID and opening a bank account.\n',
   'response': 'utter_greet'},
  {'examples': '- To get started, may I know your name?\n',
   'response': 'utter_ask_name'},
  {'examples': '- Nice to meet you, (name)! How can I help?\n',
   'response': 'utter_name'},
  {'examples': '- I am a bot.\n', 'response': 'utter_bot_challenge'},
  {'examples': '- Is there anything else I can help you with?\n',
   'response': 'utter_further_assistance'},
  {'examples': '- Hope I was able to help. Bye, bye!\n',
   'response': 'utter_goodbye'},
  {'examples': "- I'm sorry! We don't have information on that specific topic. You can ask me about the following:\n\n1. First Time Job Seeker Assistance\n2. Onboarding Requirements\n2.1. NBI Clearance\n2.2. Police Clearance\n2.3. Medical Certificate\n2.4. PSA Birth Certificate\n2.5. SSS Number\n2.6. PhilHealth Number\n2.7. PagI

**Step 4: Define Entities and Slots**

**Step 5: Create Stories**

**Step 6: Implement Custom Actions**

**Step 7: Create additional rules to hande out of bounds scenario**

In [ ]:
!pip install pyyaml

**Step 7: Train and Test Your Model**

In [158]:
!rasa train --domain /content/domain.yml

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe

In [155]:
!rasa data validate

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe

In [157]:
!rasa shell

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe